In [4]:
import numpy as np
import random
import time
import matplotlib.pyplot as plt
import shamirs

# Helper function to convert a binary string to bytes
def bin_str_to_bytes(bin_str):
    return int(bin_str, 2).to_bytes((len(bin_str) + 7) // 8, byteorder='big')

# Helper function to convert bytes back to a binary string
def bytes_to_bin_str(b):
    return bin(int.from_bytes(b, byteorder='big'))[2:].zfill(len(b) * 8)

# XOR-based Secret Sharing using bytes
def encode(secret):
    M = bin(secret)[2:]
    if len(M) % 2 != 0:
        M += '0'
    half_len = len(M) // 2
    M1 = bin_str_to_bytes(M[:half_len])
    M2 = bin_str_to_bytes(M[half_len:])
    
    R1 = random.randbytes(len(M1))
    R2 = random.randbytes(len(M2))
    
    E1 = R1 + bytes(a ^ b for a, b in zip(M2, R2))
    E2 = bytes(a ^ b for a, b in zip(M1, R1)) + R2
    E3 = bytes(a ^ b for a, b in zip(M1, R2)) + bytes(a ^ b for a, b in zip(M2, R1))
    
    return E1, E2, E3

def decode(part1, part2, E1, E2, E3):
    half_len = len(part1) // 2

    if part1 == E1 and part2 == E2:
        R1 = part1[:half_len]
        R2 = part2[half_len:]
        M1 = bytes(a ^ b for a, b in zip(part2[:half_len], R1))
        M2 = bytes(a ^ b for a, b in zip(part1[half_len:], R2))
    elif part1 == E1 and part2 == E3:
        R1 = part1[:half_len]
        M2 = bytes(a ^ b for a, b in zip(part2[half_len:], R1))
        R2 = bytes(a ^ b for a, b in zip(part1[half_len:], M2))
        M1 = bytes(a ^ b for a, b in zip(part2[:half_len], R2))
    elif part1 == E2 and part2 == E3:
        R2 = part1[half_len:]
        M1 = bytes(a ^ b for a, b in zip(part2[:half_len], R2))
        R1 = bytes(a ^ b for a, b in zip(part1[:half_len], M1))
        M2 = bytes(a ^ b for a, b in zip(part2[half_len:], R1))
    else:
        raise ValueError("Invalid parts provided for decoding.")

    M = M1 + M2
    return int.from_bytes(M, byteorder='big')

# Testing with the same loop as before
iterations = 100000

# Arrays to store runtimes
encoder_runtimes_shamir = []
decoder_runtimes_shamir = []
encoder_runtimes_xor = []
decoder_runtimes_E1_E2 = []
decoder_runtimes_E1_E3 = []
decoder_runtimes_E2_E3 = []

secret = 12345
n = 3
k = 2

for _ in range(iterations):
    # Shamir's Scheme
    start_time = time.time()
    shares = shamirs.shares(secret, quantity=3, threshold=2)
    encoder_runtimes_shamir.append((time.time() - start_time) * 1000000)  # Convert to microseconds

    start_time = time.time()
    recovered_secret = shamirs.interpolate(shares)
    decoder_runtimes_shamir.append((time.time() - start_time) * 1000000)  # Convert to microseconds

    # XOR-based Secret Sharing using bytes
    start_time = time.time()
    E1, E2, E3 = encode(secret)
    encoder_runtimes_xor.append((time.time() - start_time) * 1000000)  # Convert to microseconds

    start_time = time.time()
    decode(E1, E2, E1, E2, E3)
    decoder_runtimes_E1_E2.append((time.time() - start_time) * 1000000)  # Convert to microseconds

    start_time = time.time()
    decode(E1, E3, E1, E2, E3)
    decoder_runtimes_E1_E3.append((time.time() - start_time) * 1000000)  # Convert to microseconds

    start_time = time.time()
    decode(E2, E3, E1, E2, E3)
    decoder_runtimes_E2_E3.append((time.time() - start_time) * 1000000)  # Convert to microseconds

# Calculating the averages
avg_encoder_runtime_shamir = np.mean(encoder_runtimes_shamir)
avg_decoder_runtime_shamir = np.mean(decoder_runtimes_shamir)
avg_encoder_runtime_xor = np.mean(encoder_runtimes_xor)
avg_decoder_runtime_E1_E2 = np.mean(decoder_runtimes_E1_E2)
avg_decoder_runtime_E1_E3 = np.mean(decoder_runtimes_E1_E3)
avg_decoder_runtime_E2_E3 = np.mean(decoder_runtimes_E2_E3)

# Calculate speedups
encoder_speedup = avg_encoder_runtime_shamir / avg_encoder_runtime_xor
decoder_speedup_E1_E2 = avg_decoder_runtime_shamir / avg_decoder_runtime_E1_E2
decoder_speedup_E1_E3 = avg_decoder_runtime_shamir / avg_decoder_runtime_E1_E3
decoder_speedup_E2_E3 = avg_decoder_runtime_shamir / avg_decoder_runtime_E2_E3

print(f"Encoder speedup (XOR vs Shamir): {encoder_speedup:.2f}x")
print(f"Decoder speedup (E1,E2 vs Shamir): {decoder_speedup_E1_E2:.2f}x")
print(f"Decoder speedup (E1,E3 vs Shamir): {decoder_speedup_E1_E3:.2f}x")
print(f"Decoder speedup (E2,E3 vs Shamir): {decoder_speedup_E2_E3:.2f}x")

# Plotting the results
plt.figure(figsize=(14, 7), dpi=100)

plt.subplot(1, 2, 1)
plt.bar(['Shamir', 'XOR'], [avg_encoder_runtime_shamir, avg_encoder_runtime_xor], color=['#1f77b4', '#ff7f0e'])
plt.title('Average Encoding Runtime (2, 3)', fontsize=24)
plt.ylabel('Time (microseconds)', fontsize=24)
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)

plt.subplot(1, 2, 2)
plt.bar(['Shamir', 'E1,E2', 'E1,E3', 'E2,E3'],
        [avg_decoder_runtime_shamir, avg_decoder_runtime_E1_E2, avg_decoder_runtime_E1_E3, avg_decoder_runtime_E2_E3],
        color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
plt.title('Average Decoding Runtime (2, 3)', fontsize=24)
plt.ylabel('Time (microseconds)', fontsize=24)
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)

plt.tight_layout()
plt.show()

# Plotting the speedup
plt.figure(figsize=(10, 7), dpi=100)
plt.bar(['Encoder', 'E1,E2', 'E1,E3', 'E2,E3'],
        [encoder_speedup, decoder_speedup_E1_E2, decoder_speedup_E1_E3, decoder_speedup_E2_E3],
        color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
plt.title('Speedup of XOR-based Scheme Over Shamir\'s Scheme', fontsize=24)
plt.ylabel('Speedup Factor', fontsize=24)
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)

plt.tight_layout()
plt.show()



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/abhisekjha/Library/Python/3.11/lib/python/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/abhisekjha/Library/Python/3.11/lib/python/site-packages/traitlets/config/application.py", line 1043, in launch_instance
    app.start()
  File "/Users/abhisekjha/Library/Python/3.11/lib/python/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [ ]:
# Plotting the results
plt.figure(figsize=(14, 7), dpi=100)

plt.subplot(1, 2, 2)
plt.bar(['Shamir', 'E1,E2', 'E1,E3', 'E2,E3'],
        [avg_decoder_runtime_shamir, avg_decoder_runtime_E1_E2, avg_decoder_runtime_E1_E3, avg_decoder_runtime_E2_E3],
        color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
plt.title('Average Decoding Runtime (2, 3)', fontsize=24)
plt.ylabel('Time (microseconds)', fontsize=24)
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)

plt.tight_layout()
plt.show()